# Metaheuristics parameter tuning using MetaGA

This example is a tutorial and a showcase of the MSA framework capabilities and features for tuning metaheuristics parameters.

### Import tools and libraries

In [ ]:
# Analysis tools
from msa.tools.meta_ga import MetaGAFitnessFunction, MetaGA

# Optimization problem from the NiaPy framework
from niapy.problems.schwefel import Schwefel

# Other tools
import os
import pprint
import matplotlib
matplotlib.use("inline")

# Define constants for later use
base_archive_path = "archive"
meta_ga_pkl_filename = "meta_ga_example"

### Define gene spaces used for the parameter tuning
Name of the algorithm can be a string corresponding with the name of the algorithm class from the `niapy.algorithms` framework, or a class inheriting from the niapy `Algorithm` class. Names of the hyperparameters must correspond with the names of selected algorithm constructor arguments.

---
**_TIP:_** In the following step are some gene space examples to use in the following code. Try defining gene spaces for other algorithms from niapy framework.

---

In [ ]:
from msa.algorithms.pso import ParticleSwarmAlgorithm # Modified using `limit` repair
from msa.algorithms.fa import FireflyAlgorithm # Modified for faster processing with help of @jit and reduction of evaluations per generation
from niapy.algorithms.basic.de import DifferentialEvolution
from niapy.algorithms.basic.bea import BeesAlgorithm

PSO_gene_space = {
    ParticleSwarmAlgorithm: {
        "c1": {"low": 0.01, "high": 2.5, "step": 0.01},
        "c2": {"low": 0.01, "high": 2.5, "step": 0.01},
        "w": {"low": 0.0, "high": 1.0, "step": 0.01},
    }
}

BA_gene_space = {
    "BatAlgorithm": {
        "loudness": {"low": 0.01, "high": 1.0, "step": 0.01},
        "pulse_rate": {"low": 0.01, "high": 1.0, "step": 0.01},
        "alpha": {"low": 0.9, "high": 1.0, "step": 0.001},
        "gamma": {"low": 0.0, "high": 1.0, "step": 0.01},
    }
}

FA_gene_space = {
    FireflyAlgorithm: {
        "alpha": {"low": 0.01, "high": 1.0, "step": 0.01},
        "beta0": {"low": 0.01, "high": 1.0, "step": 0.01},
        "gamma": {"low": 0.0, "high": 1.0, "step": 0.001},
        "theta": {"low": 0.95, "high": 1.0, "step": 0.001},
    }
}

DE_gene_space = {
    DifferentialEvolution: {
        "differential_weight": {"low": 0.01, "high": 1.0, "step": 0.01},
        "crossover_probability": {"low": 0.01, "high": 1.0, "step": 0.01},
    }
}


### Choose the optimization problem used for the parameter tuning

In [ ]:
OPTIMIZATION_PROBLEM = Schwefel(dimension=20)

### Define a MetaGA instance and start tuning
* Set `fitness_function_type` to `MetaGAFitnessFunction.TARGET_PERFORMANCE_SIMILARITY`.
* Arguments starting with `ga_` correspond with the arguments of the class `GA` from `pygad` module.
* Set `gene_space` to gene space of the algorithm to be optimized.

---
**_TIP:_** Try out gene spaces defined in previous step or add new ones to tune their parameters.

---
**_NOTE:_** Use different `prefix` value for future experiments to prevent overwriting the existing export. It can also be left unspecified to use the current datetime value.

---

In [ ]:
meta_ga = MetaGA(
    fitness_function_type=MetaGAFitnessFunction.PARAMETER_TUNING,
    ga_generations=3,
    ga_solutions_per_pop=5,
    ga_percent_parents_mating=60,
    ga_parent_selection_type="tournament",
    ga_k_tournament=2,
    ga_crossover_type="uniform",
    ga_mutation_type="random",
    ga_crossover_probability=0.9,
    ga_mutation_num_genes=1,
    ga_keep_elitism=1,
    gene_space=FA_gene_space,
    pop_size=30,
    max_evals=10000,
    num_runs=30,
    problem=OPTIMIZATION_PROBLEM,
    base_archive_path=base_archive_path,
)

best_solution = meta_ga.run_meta_ga(
    get_info=True, prefix="tuning_example", export=True, pkl_filename=meta_ga_pkl_filename
)

### Apply the best solution
The best solution of MetaGA can be applied to the algorithm via `MetaGA.solution_to_algorithm_attributes` method.

In [ ]:
algorithm = MetaGA.solution_to_algorithm_attributes(solution=best_solution, gene_space=meta_ga.gene_space, pop_size=30)
pprint.pprint(algorithm.get_parameters())

### Import MetaGA instance

Setting `export` argument to `True` in the previous step enables importing via the static method `MetaGA.import_from_pkl`.

---
**_NOTE:_** Make sure to provide the correct path to the exported .pkl file.

---

In [ ]:
pkl_path = os.path.join(base_archive_path, "tuning_example_FA_Schwefel", meta_ga_pkl_filename)
print(pkl_path)
imported_meta_ga = MetaGA.import_from_pkl(pkl_path)

### Plot MetaGA solution evolution
Call `plot_solutions` to create a plot showing solutions of MetaGA trough generations:
* Set `all_solutions` to `True` to plot all solutions or leave it at default value `False` to plot only the best solutions of each generation.

Plots will be saved under the corresponding directory.

In [ ]:
imported_meta_ga.plot_solutions(filename="meta_ga_all_solutions_evolution", all_solutions=True)

In [ ]:
imported_meta_ga.plot_solutions(filename="meta_ga_best_solutions_evolution")

### Plot MetaGA fitness
Call `plot_fitness` to create a plot showing the fitness of the best solution found by MetaGA in each generation:
Fitness is calculated as fitness of the best solution for the optimization problem found by the metaheuristic being tuned, to the power of -1.

Plot will be saved under the corresponding directory.

In [ ]:
imported_meta_ga.plot_fitness(filename="meta_ga_fitness_plot")